**In/Out 공통함수**

In [ ]:
# 추가한 부분!!!!!!
def extract_inout_features(packet_sequence):
    incoming_count = 0
    outgoing_count = 0

    for c in packet_sequence:
        if c > 0:
            outgoing_count += 1
        else:
            incoming_count += 1

    total_packets = len(packet_sequence)
    incoming_fraction = incoming_count / total_packets if total_packets > 0 else 0
    outgoing_fraction = outgoing_count / total_packets if total_packets > 0 else 0

    return incoming_count, outgoing_count, incoming_fraction, outgoing_fraction

1. mon_standard.pkl > array code



In [ ]:
import pickle

# 추가한 부분!!!!!!
def extract_inout_features(packet_sequence):
    incoming_count = sum(1 for c in packet_sequence if c < 0)
    outgoing_count = sum(1 for c in packet_sequence if c > 0)
    total_packets = len(packet_sequence)
    incoming_fraction = incoming_count / total_packets if total_packets > 0 else 0
    outgoing_fraction = outgoing_count / total_packets if total_packets > 0 else 0
    return incoming_count, outgoing_count, incoming_fraction, outgoing_fraction


USE_SUBLABEL = False
URL_PER_SITE = 10
TOTAL_URLS   = 950

# Load the pickle file
print("Loading datafile...")
with open("/content/mon_standard.pkl", 'rb') as fi: # Path to mon_standard.pkl in Colab
    data = pickle.load(fi)

X1 = [] # Array to store instances (timestamps) - 19,000 instances, e.g., [[0.0, 0.5, 3.4, ...], [0.0, 4.5, ...], [0.0, 1.5, ...], ... [... ,45.8]]
X2 = [] # Array to store instances (direction*size) - size information
y = [] # Array to store the site of each instance - 19,000 instances, e.g., [0, 0, 0, 0, 0, 0, ..., 94, 94, 94, 94, 94]
features_list = []

# Differentiate instances and sites, and store them in the respective x and y arrays
# x array (direction*timestamp), y array (site label)
for i in range(TOTAL_URLS):
    if USE_SUBLABEL:
        label = i
    else:
        label = i // URL_PER_SITE # Calculate which site's URL the current URL being processed belongs to and set that value as the label. Thus, URLs fetched from the same site are labeled identically.
    for sample in data[i]:
        size_seq = []
        time_seq = []
        for c in sample:
            dr = 1 if c > 0 else -1
            time_seq.append(abs(c))
            size_seq.append(dr * 512)
        X1.append(time_seq)
        X2.append(size_seq)
        y.append(label)

        # 추가한 부분!!!!!!
        # Extract in/out features
        in_count, out_count, in_frac, out_frac = extract_inout_features(size_seq)
        features_list.append([in_count, out_count, in_frac, out_frac])
size = len(y)

print(f'Total samples: {size}') # Output: 19000


Loading datafile...
Total samples: 19000


2. unmon_standard10.pkl > array code

In [ ]:
import pickle
import numpy as np

# 추가한 부분!!!!!!
def extract_inout_features(packet_sequence):
    incoming_count = sum(1 for c in packet_sequence if c < 0)
    outgoing_count = sum(1 for c in packet_sequence if c > 0)
    total_packets = len(packet_sequence)
    incoming_fraction = incoming_count / total_packets if total_packets > 0 else 0
    outgoing_fraction = outgoing_count / total_packets if total_packets > 0 else 0
    return incoming_count, outgoing_count, incoming_fraction, outgoing_fraction

TOTAL_URLS = 10000  # total number in the dataset

# Load 10,000 unmon pickle file
print("Loading datafile...")
with open('/content/unmon_standard10.pkl', 'rb') as f:  # Path to unmon_standard10.pkl in Colab
    x = pickle.load(f)

size = len(x)
print(f'Total samples: {size}')

X1 = [] # Array to store instances (timestamps) - 10,000 instances, e.g., [[0.0, 0.5, 3.4, ...], [0.0, 4.5, ...], [0.0, 1.5, ...], ... [... ,45.8]]
X2 = [] # Array to store instances (direction*size) - size information
features_list = []

for i in range(TOTAL_URLS):
    size_seq = []
    time_seq = []
    for c in x[i]:
        dr = 1 if c > 0 else -1
        time_seq.append(abs(c))
        size_seq.append(dr * 512) # In the pickle file, there is no size information, so the conversion code is set to multiply by 512 uniformly.
    X1.append(time_seq)
    X2.append(size_seq)

    # 추가한 부분!!!!!
    # Extract features from size_seq
    in_count, out_count, in_frac, out_frac = extract_inout_features(size_seq)
    features_list.append([in_count, out_count, in_frac, out_frac])

print(len(X1)) # Print the length of X1

# Convert to numpy arrays
features_unmon = np.array(features_list)
labels_unmon = np.full(len(X1), -1)  # Unmonitored label: -1

# 추가한 부분!!!!!!
print(f'Unmonitored features shape: {features_unmon.shape}')
print(f'Unmonitored labels shape: {labels_unmon.shape}')

Loading datafile...
Total samples: 10000
10000
Unmonitored features shape: (10000, 4)
Unmonitored labels shape: (10000,)
